In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', 150)
dataset = pd.read_csv("clean_bbc_news_dataset", index_col=0)
print(dataset.shape)
dataset.head()

(2225, 2)


,category,clean_text
0,tech,"['tv', 'futur', 'hand', 'viewer', 'home', 'theatr', 'system', 'plasma', 'highdefinit', 'tv', 'digit', 'video', 'record', 'move', 'live', 'room', '..."
1,business,"['worldcom', 'boss', 'left', 'book', 'alon', 'former', 'worldcom', 'boss', 'berni', 'ebber', 'accus', 'overse', '11bn', '£58bn', 'fraud', 'never',..."
2,sport,"['tiger', 'wari', 'farrel', 'gambl', 'leicest', 'say', 'rush', 'make', 'bid', 'andi', 'farrel', 'great', 'britain', 'rugbi', 'leagu', 'captain', '..."
3,sport,"['yead', 'face', 'newcastl', 'fa', 'cup', 'premiership', 'side', 'newcastl', 'unit', 'face', 'trip', 'ryman', 'premier', 'leagu', 'leader', 'yead'..."
4,entertainment,"['ocean', 'twelv', 'raid', 'box', 'offic', 'ocean', 'twelv', 'crime', 'caper', 'sequel', 'star', 'georg', 'clooney', 'brad', 'pitt', 'julia', 'rob..."


In [3]:
from sklearn import model_selection 

X = dataset['clean_text']
y = dataset['category']

train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, test_size=.20)
print(f'{len(train_X)} Training Sample  AND  {len(test_y)} testing Sample')
print(dataset['category'].unique())

1780 Training Sample  AND  445 testing Sample
['tech' 'business' 'sport' 'entertainment' 'politics']


# demo document first

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentence = [
    ["i" , "love", "my", "dog"],
    ["i", "love", "my", "cat"], 
    ["you", "love", "my", "dog"],
    ["do", "you", "think", "my", "dog", "is", "amazing"]
]

In [5]:
token = Tokenizer(num_words=11 , oov_token='<OOV>')
token.fit_on_texts(sentence)
print(token.word_index)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}


In [7]:
import numpy as np 
sequence = token.texts_to_sequences(sentence)

In [8]:
sequence_tfidf = token.texts_to_matrix(sentence, mode='tfidf')
print(sequence_tfidf)

[[0.         0.         0.58778666 0.69314718 0.69314718 0.84729786
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.58778666 0.69314718 0.         0.84729786
  0.         1.09861229 0.         0.         0.        ]
 [0.         0.         0.58778666 0.69314718 0.69314718 0.
  0.84729786 0.         0.         0.         0.        ]
 [0.         1.60943791 0.58778666 0.         0.69314718 0.
  0.84729786 0.         1.09861229 1.09861229 1.09861229]]


In [9]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# tfidf_calculator = TextVectorization(
#   standardize = 'lower_and_strip_punctuation',
#   split       = 'whitespace',
#   max_tokens  = 100,
#   output_mode ='tf-idf',
#   pad_to_max_tokens=False)

In [10]:
# tfidf_calculator.adapt(sentence)
# tfidf_calculator.get_vocabulary()
# tfids = tfidf_calculator(sentence)
# print(tfids)

# using main dataset

In [6]:
# word lenth for each coloum
import numpy as np
w_count = []
def get_word_lenght(text):
    w_count.append(len(text))
    return len(text)
    
dataset['word_count'] = dataset['clean_text'].apply(lambda x:get_word_lenght(x))
max_val = dataset['word_count'].max()
print('MAX_VAL ', max_val)
w = np.array(w_count).mean()
w

MAX_VAL  20861


2056.2260674157305

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection

In [9]:
label_encode = LabelEncoder()
sentence = dataset['clean_text']
label = label_encode.fit_transform(dataset['category'])
label

array([4, 0, 3, ..., 1, 2, 3])

In [12]:
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(sentence, label, test_size=.20)

In [13]:
print('X train' , len(xtrain))
print('Y train' , len(ytrain))
print('X test' , len(xtest))
print('Y test' , len(ytest))

X train 1780
Y train 1780
X test 445
Y test 445


In [14]:
# tokenizaing text ...
tokenizer = Tokenizer(num_words=25000, oov_token="<OOV>")
tokenizer.fit_on_texts(xtrain)
w_index = tokenizer.word_index
print('vocab_size =================>>>>>>>>>>>>  ' , len(w_index))
# print(w_index)

vocab_size =================>>>>>>>>>>>>   22101


In [15]:
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)
# print(xtest_seq[:3])

In [16]:
max_val = 5000
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_val)
xtest_pad = pad_sequences(xtest_seq, maxlen=max_val)
print('x train pad shape', xtrain_pad.shape)
print('x test pad shape', xtest_pad.shape)

x train pad shape (1780, 5000)
x test pad shape (445, 5000)


In [54]:
vocab_size = 25000
embedding_dim = 100
pad_length =5000

model = tf.keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=pad_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(5, activation='sigmoid')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5000, 100)         2500000   
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 500)               50500     
                                                                 
 dense_3 (Dense)             (None, 5)                 2505      
                                                                 
Total params: 2,553,005
Trainable params: 2,553,005
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.fit(xtrain_pad, ytrain , epochs=10, validation_data=(xtest_pad, ytest), verbose=1)

Epoch 1/10
56/56 [==============================] - 10s 124ms/step - loss: 1.6035 - accuracy: 0.2348 - val_loss: 1.6050 - val_accuracy: 0.2045
Epoch 2/10
56/56 [==============================] - 7s 118ms/step - loss: 1.6003 - accuracy: 0.2360 - val_loss: 1.6016 - val_accuracy: 0.2045
Epoch 3/10
56/56 [==============================] - 7s 116ms/step - loss: 1.5948 - accuracy: 0.2326 - val_loss: 1.5967 - val_accuracy: 0.2045
Epoch 4/10
56/56 [==============================] - 6s 116ms/step - loss: 1.5735 - accuracy: 0.2522 - val_loss: 1.5754 - val_accuracy: 0.2067
Epoch 5/10
56/56 [==============================] - 7s 117ms/step - loss: 1.4773 - accuracy: 0.4185 - val_loss: 1.3875 - val_accuracy: 0.5236
Epoch 6/10
56/56 [==============================] - 7s 117ms/step - loss: 1.1956 - accuracy: 0.5978 - val_loss: 1.0547 - val_accuracy: 0.6270
Epoch 7/10
56/56 [==============================] - 7s 117ms/step - loss: 0.8603 - accuracy: 0.8022 - val_loss: 0.7600 - val_accuracy: 0.8966
Epoch

In [56]:
model.evaluate(xtest_pad, ytest)

14/14 [==============================] - 1s 36ms/step - loss: 0.3019 - accuracy: 0.9438


[0.3019077479839325, 0.9438202381134033]

In [25]:
max_val = 5000
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_val)
xtest_pad = pad_sequences(xtest_seq, maxlen=max_val)
print('x train pad shape', xtrain_pad.shape)
print('x test pad shape', xtest_pad.shape)

x train pad shape (1780, 5000)
x test pad shape (445, 5000)


In [1]:
vocab_size = 25000
embedding_dim = 50
pad_length =5000

lstm_model = tf.keras.Sequential([
    
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=pad_lenght),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.5)),
    tf.keras.layers.LSTM(50, dropout=0.2),

    
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(5, activation='sigmoid')
    
])


lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
lstm_model.summary()


NameError: name 'tf' is not defined

In [28]:
lstm_model.fit(xtrain_pad, ytrain , epochs=7, validation_data=(xtest_pad, ytest), verbose=1)

Epoch 1/7
56/56 [==============================] - 393s 7s/step - loss: 1.5172 - accuracy: 0.2893 - val_loss: 1.3081 - val_accuracy: 0.4180
Epoch 2/7
56/56 [==============================] - 400s 7s/step - loss: 1.0418 - accuracy: 0.5129 - val_loss: 0.9496 - val_accuracy: 0.5640
Epoch 3/7
56/56 [==============================] - 425s 8s/step - loss: 0.6771 - accuracy: 0.6573 - val_loss: 0.8192 - val_accuracy: 0.5596
Epoch 4/7
56/56 [==============================] - 389s 7s/step - loss: 0.5488 - accuracy: 0.7697 - val_loss: 0.6805 - val_accuracy: 0.6382
Epoch 5/7
56/56 [==============================] - 372s 7s/step - loss: 0.3472 - accuracy: 0.8843 - val_loss: 0.4932 - val_accuracy: 0.8629
Epoch 6/7
56/56 [==============================] - 337s 6s/step - loss: 0.2613 - accuracy: 0.9270 - val_loss: 0.6224 - val_accuracy: 0.7933
Epoch 7/7
56/56 [==============================] - 394s 7s/step - loss: 0.1164 - accuracy: 0.9792 - val_loss: 0.4064 - val_accuracy: 0.8742


In [29]:
lstm_model.evaluate(xtest_pad, ytest)

14/14 [==============================] - 7s 520ms/step - loss: 0.4064 - accuracy: 0.8742


[0.40643972158432007, 0.8741573095321655]

In [34]:
tokenizer2 = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer2.fit_on_texts(xtrain)
w_index2 = tokenizer2.word_index
print('vocab_size' , len(w_index2))
# print(w_index2)

vocab_size 21845


In [35]:
xtrain_seq2 = tokenizer2.texts_to_matrix(xtrain, mode='tfidf')
xtest_seq2 = tokenizer2.texts_to_matrix(xtest, mode='tfidf')

print(xtrain_seq2.shape)

(1780, 5000)


In [45]:
max_val = 200
xtrain_pad2 = pad_sequences(xtrain_seq2, maxlen=max_val)
xtest_pad2 = pad_sequences(xtest_seq2, maxlen=max_val)
print('x train pad shape', xtrain_pad2.shape)
print('x test pad shape', xtest_pad2.shape)

x train pad shape (1780, 200)
x test pad shape (445, 200)


In [48]:
vocab_size = 5000
w_dim = 128
pad_size = max_val

tf_idf_model = keras.Sequential([
    keras.layers.Embedding(vocab_size , w_dim),
    keras.layers.Bidirectional(keras.layers.LSTM(128)),
    
    keras.layers.Dense(5, activation='sigmoid')
])

tf_idf_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tf_idf_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 128)         640000    
                                                                 
 bidirectional_8 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_12 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 904,453
Trainable params: 904,453
Non-trainable params: 0
_________________________________________________________________


In [50]:
# tf_idf_model.fit(xtrain_pad2, ytrain , epochs=28, validation_data=(xtest_pad2, ytest), verbose=1)

In [40]:
tf_idf_model.evaluate(xtest_pad2, ytest)

14/14 [==============================] - 1s 63ms/step - loss: 1.5998 - accuracy: 0.2090


[1.599849820137024, 0.20898877084255219]

{'<OOV>': 1,
 'my': 2,
 'love': 3,
 'dog': 4,
 'i': 5,
 'you': 6,
 'cat': 7,
 'do': 8,
 'think': 9,
 'is': 10,
 'amazing': 11}

[[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]

array([[ 0,  0,  0,  5,  3,  2,  4],
       [ 0,  0,  0,  5,  3,  2,  7],
       [ 0,  0,  0,  6,  3,  2,  4],
       [ 8,  6,  9,  2,  4, 10, 11]])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 7, 4)              400       
                                                                 
 flatten_2 (Flatten)         (None, 28)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 29        
                                                                 
Total params: 429
Trainable params: 429
Non-trainable params: 0
_________________________________________________________________


In [61]:
x = pad_token
y = np.array([0,0,1,1])
mod.fit(x,y, epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6920 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6906 - accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6893 - accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6879 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6865 - accuracy: 0.2500
Epoch 6/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6851 - accuracy: 0.7500
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6838 - accuracy: 0.7500
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6824 - accuracy: 0.7500
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6811 - accuracy: 0.7500
Epoch 10/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6797 - accuracy: 0.7500
Epoch 11/50
1/1 [======================

1/1 [==============================] - 0s 22ms/step - loss: 0.6227 - accuracy: 0.7500


[0.6226552724838257, 0.75]

In [64]:
weights[7]

array([-0.02007433, -0.05071082,  0.00803106, -0.08737522], dtype=float32)

array([-0.03500351, -0.10022003, -0.00099805, -0.0312709 ], dtype=float32)